![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b> Chat & Memory </b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()
# Bei Bedarf: Trennen zwischen Installationsname () und Importname (für Python) beide Angaben in Klammern
# install_packages([('markitdown[all]', 'markitdown'), 'langchain_chroma', ]

In [ ]:
#@title 🛠️ Installationen{ display-mode: "form" }
install_packages(['chromadb', ])

# 1 | Intro
---


Ein Gespräch mit jemandem zu führen, der nach jedem Satz vergisst, was zuvor gesagt wurde, wäre äußerst frustrierend. Genau dieses Problem löst **Memory** in der Künstlichen Intelligenz.

**Warum braucht KI ein Gedächtnis?**

Large Language Models wie GPT sind von Natur aus **zustandslos** – sie verfügen über kein eingebautes Gedächtnis. Jede Anfrage wird isoliert verarbeitet, ohne Bezug zu vorherigen Interaktionen. Für sinnvolle Gespräche und intelligente Assistenten ist dies jedoch unbrauchbar.


Dieses Kapitel behandelt drei fundamentale Memory-Typen – vom einfachen Zwischenspeicher bis zu ausgeklügelten Wissensdatenbanken. Dabei werden konkrete Technologien und Implementierungsansätze vorgestellt, die sich unmittelbar in eigenen Projekten einsetzen lassen.



Die folgenden Abschnitte führen systematisch durch die Welt von Chat & Memory und deren praktische Anwendung.


[Kontextfenster](https://editor.p5js.org/ralf.bendig.rb/full/tLnUgyZRK)

| Typ               | Beschreibung                  | Beispiel                                                           | Technologie - Beispiele                                                     | Speicherort      |
| ----------------- | ----------------------------- | ------------------------------------------------------------------ | --------------------------------------------------------------------------- | ---------------- |
| Kurzzeit-Memory<br> (temporär) | Innerhalb einer Sitzung       | ChatGPT erinnert sich an das, was du vor 3 Nachrichten gesagt hast | `Python Liste`<br>`ConversationBufferMemory`<br>`ConversationSummaryMemory` | nur im RAM       |
| Langzeit-Memory   | Über mehrere Sitzungen hinweg | KI merkt sich deinen Namen, Interessen etc.                        | JSON-Datei + `ConversationChain`                                            | Festplatte       |
| Externes Memory   | Via Datenbanken, Dateien etc. | RAG-Systeme, Notizsysteme                                          | `Chroma` + `RetrievalQA`                                                    | Wissensdatenbank |

# 2 | Kurzzeit-Memory
---

Temporäres Memory bildet die Grundlage jeder KI-Konversation. Es speichert den unmittelbaren Gesprächsverlauf einer Sitzung und ermöglicht es der KI, auf vorherige Nachrichten Bezug zu nehmen.
Funktionsweise
Das System hält die letzten Nachrichten im Arbeitsspeicher vor und fügt sie bei jeder neuen Anfrage als Kontext hinzu. Dadurch entsteht der Eindruck eines zusammenhängenden Gesprächs, obwohl das zugrundeliegende Modell weiterhin zustandslos arbeitet.

## 2.1 | Python Liste

In [ ]:
# Importe
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
# Konstanten & Propmpt-Template
model_name = "gpt-4o-mini"
temperature = 0

system_prompt = "Du bist ein hilfreicher und humorvoller KI-Assistent"

# Die Chat-Prompt definieren mit drei Variablen: system_prompt, history, question
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [ ]:
# LLM definieren
llm = ChatOpenAI(model=model_name, temperature=temperature)

# Parser
parser = StrOutputParser()

# Die Konversationskette definieren
chain = prompt | llm | parser

In [ ]:
# Funktion definieren
def interact_with_ai(input, history):
    """Führt eine einzelne Interaktion mit der KI durch."""

    # -- Aufruf der Kette
    response = chain.invoke(
        {"system_prompt": system_prompt, "history": history, "input": input}
    )

    # -- Ausgabe
    mprint("### 🧑‍🦱 Mensch:")
    question = input.replace("\n", "<br>")
    mprint(input)

    mprint("### 🤖 KI:")
    mprint(response)

    # -- Memory-Mangament
    history.extend([HumanMessage(content=question), AIMessage(content=response)])

In [ ]:
# Abschnitt 5: Hauptprogramm

# Historie wird initialisiert
history = [SystemMessage(content=system_prompt)]

# Liste mit user-input
user_input = "Mein Name ist Ralf"
interact_with_ai(user_input, history)

# Liste mit user-input
user_input = "Hast Du Dir meinen Namen gemerkt?"
interact_with_ai(user_input, history)

In [ ]:
# Python Liste
history

## 2.2 | CoversationBufferMemory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
# Temporärer Speicher & Prompt-Template & Parser
memory = ConversationBufferMemory(return_messages=True)

# Prompt-Vorlage mit Platzhalter für den bisherigen Chat-Verlauf
prompt = ChatPromptTemplate.from_messages([
    ('system', "Du bist ein hilfreicher und humorvoller KI-Assistent"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Parser
parser = StrOutputParser()

In [ ]:
# LLM & Chain definieren
model_name = "gpt-4o-mini"
temperature = 0
llm = ChatOpenAI(model=model_name, temperature=temperature)

chain = prompt | llm | parser

In [ ]:
# Beispiel-Interaktionen
# 1. Eingabe speichern
response = chain.invoke({
    "input": "Hallo, ich bin Ralf.",
    "history": memory.chat_memory.messages
})

mprint(response)

memory.chat_memory.add_user_message("Hallo, ich bin Ralf.")
memory.chat_memory.add_ai_message(response)

In [ ]:
# 2. Folgeeingabe
response = chain.invoke({
    "input": "Weißt du noch, wie ich heiße?",
    "history": memory.chat_memory.messages
})

mprint(response)

memory.chat_memory.add_user_message("Weißt du noch, wie ich heiße?")
memory.chat_memory.add_ai_message(response)

In [ ]:
# Memory
memory.chat_memory.messages

## 2.3 | ConversationSummaryMemory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
# Memory & Propmt-Template & Parser
memory = ConversationSummaryMemory(llm=llm, return_messages=True)

# Prompt mit history-Placeholder!
prompt = ChatPromptTemplate.from_messages([
    ('system', "Du bist ein hilfreicher und humorvoller KI-Assistent."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Parser
parser = StrOutputParser()

In [ ]:
# LLM & Chain
model_name = "gpt-4o-mini"
temperature = 0
llm = ChatOpenAI(model=model_name, temperature=temperature)

# Chain (manuell gebaut)
chain = prompt | llm | parser

In [ ]:
# Interaktion mit Memory-Nutzung
def interact_with_ai(question):
    # Verlauf laden
    history_vars = memory.load_memory_variables({})

    # Ausgabe anzeigen
    mprint(f"###🧑‍🦱 Mensch:\n {question}")

    # Chain mit Kontext aus Memory aufrufen
    response = chain.invoke({
        "input": question,
        "history": history_vars["history"]
    })

    mprint(f"### 🤖 KI:\n {response}\n")

    # neuen Dialog abspeichern
    memory.save_context({"input": question}, {"output": response})

In [ ]:
# Test-Dialog
user_input = [
    "Mein Name ist Ralf",
    "Warum ist der Himmel blau?",
    "Warum ist er manchmal rot?",
    "Wie heiße ich?"
]

for input_text in user_input:
    interact_with_ai(input_text)

In [ ]:
# Summary
memory.load_memory_variables({})['history']

# 3 | Langzeit-Memory
---

Ein Langzeit-Memory ermöglicht es einer KI, über mehrere Sitzungen hinweg Informationen zu behalten – ähnlich wie ein Mensch sich an frühere Gespräche oder Fakten erinnert.

In [ ]:
import json
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Funktionen
def chat():
    """Hauptfunktion für den Chat"""
    # Memory und Chain setup
    memory = ConversationBufferMemory(return_messages=True)
    memory.chat_memory.messages = load_memory()

    chain = (ChatPromptTemplate.from_messages([
        ("system", "Du bist ein hilfreicher Assistent."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]) | ChatOpenAI(temperature=0) | StrOutputParser())

    print("🤖 Chat gestartet. 'exit' zum Beenden.\n")

    # Chat-Schleife
    while True:
        user_input = input("🧑 Frage: ")

        if user_input.lower() in ["exit", "quit"]:
            print("👋 Tschüss!")
            break

        # Antwort generieren
        response = chain.invoke({
            "input": user_input,
            "history": memory.chat_memory.messages
        })

        print(f"🤖 {response}\n")

        # Memory aktualisieren
        memory.save_context({"input": user_input}, {"output": response})
        save_memory(memory.chat_memory.messages)

def load_memory():
    """Lädt gespeicherte Chat-Historie"""
    try:
        with open("chat_memory.json", "r") as f:
            from langchain.schema import messages_from_dict
            return messages_from_dict(json.load(f))
    except:
        return []

def save_memory(messages):
    """Speichert Chat-Historie"""
    from langchain.schema import messages_to_dict
    with open("chat_memory.json", "w") as f:
        json.dump(messages_to_dict(messages), f)

In [ ]:
# Starten
if __name__ == "__main__":
    chat()

In [ ]:
chat()

# 4 | Externes Memory
---


Ein externes Memory bedeutet, dass sich ein KI-System nicht alles selbst merken muss, sondern bei Bedarf Wissen von außen abruft. Das wird häufig mit einer Vektor-Datenbanken umgesetzt – das Herzstück vieler moderner Chatbots mit „Langzeitwissen“.

Eine Vektordatenbank speichert Texte, Bilder oder andere Inhalte in einer numerischen Form (sogenannte Embeddings), damit sie schnell durchsucht werden können.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# Datenbank vorbereiten
texts = [
    "Python ist eine vielseitige Programmiersprache.",
    "Künstliche Intelligenz bezeichnet die Simulation menschlicher Intelligenz durch Maschinen."
]

# Embedding & Chroma-Vektordatenbank
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_texts(texts, embedding=embedding, persist_directory="chroma_db")

In [ ]:
# LLM + Retriever
model_name = "gpt-4o-mini"
temperature = 0
llm = ChatOpenAI(model=model_name, temperature=temperature)

retriever = vectordb.as_retriever()

In [ ]:
# RetrievalQA-Chain mit invoke-kompatiblem Format
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

In [ ]:
# Beispiel-Konversation mit invoke()
fragen = [
    "Was ist Python?",
    "Und was ist KI?",
    "Was weißt du über Java?"  # Nicht in Datenbank enthalten
]

for frage in fragen:
    mprint(f"###🧑‍💻 Mensch:\n {frage}")
    antwort = qa_chain.invoke({"query": frage})
    mprint(f"###🤖 KI:\n {antwort['result']}\n")

# 5 | Mehr Tokens ≠ Bessere Chat-Memory

Mehr Tokens in einem Kontextfenster eines Large Language Models (LLMs) bedeuten nicht automatisch eine bessere Verarbeitung. Dies ist besonders relevant im Kontext von **Chat & Memory**, wo Modelle versuchen, längere Gesprächsverläufe oder gespeicherte Erinnerungen zu nutzen.

1. **Rauschen und Irrelevanz**
   Ein größeres Kontextfenster erlaubt zwar die Verarbeitung längerer Chat-Historien, aber nicht alle Informationen sind für die aktuelle Anfrage relevant. Das Modell muss wichtige Details von unwichtigen trennen, was mit wachsendem Kontext schwieriger wird.

2. **Abnehmende Aufmerksamkeit**
   LLMs nutzen Aufmerksamkeitsmechanismen. Bei sehr langen Chat-Kontexten verteilt sich die Aufmerksamkeit über viele Tokens, sodass entscheidende Informationen weniger stark gewichtet werden können.

3. **Fehlerakkumulation**
   In Chats können kleine Missverständnisse oder falsche Annahmen durch längere Kontexte verstärkt werden. Dies kann zu Antworten führen, die auf alten, irrelevanten Informationen basieren.

4. **Verarbeitungsgrenzen und Latenz**
   Größere Kontextfenster führen zu mehr Rechenaufwand, längeren Antwortzeiten und höherem Ressourcenverbrauch – ohne dass die Qualität proportional steigt.

5. **Warum das Thema weiterhin wichtig ist**
   Mit der zunehmenden Nutzung von **Memory-Funktionen** in Chatbots stellt sich die Frage, wie Erinnerungen strukturiert, gefiltert und priorisiert werden. Nur durch intelligentes Kontext- und Memory-Management lassen sich wirklich relevante Informationen nutzen, ohne das Modell mit unnötigen Daten zu überfluten.


<p><font color='darkblue' size="4">
ℹ️ <b>Fazit</b>
</font></p>

Ein großes Kontextfenster allein löst nicht die Herausforderungen von Chat & Memory. Entscheidend sind clevere Strategien, um Informationen zu selektieren, zusammenzufassen und zielgerichtet einzusetzen.


# A | Aufgaben
---




Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
KI-gestütztes Notizbuch mit automatischer Kategorisierung
</font></p>


Entwickeln Sie ein KI-Notizbuch, das Eingaben von Nutzern speichert, automatisch kategorisiert (z. B. "Technologie", "Privat", "Aufgaben") und kontextbezogene Vorschläge liefert. Verwenden Sie dazu ein LangChain Memory-Konzept, z.B. ConversationBufferMemory, für eine effiziente Verwaltung.



<p><font color='black' size="5">
Chatbot: Nachrichten- vs. Summary-Speicher
</font></p>


Erstellen Sie einen einfachen Chatbot mit LangChain, der zwischen Nachrichten- und Summary-Speicher unterscheidet. Analysieren Sie, wie sich die Speichertypen auf die Qualität der Antworten auswirken.



<p><font color='black' size="5">
Automatische Protokollerstellung für Meetings
</font></p>


Entwickeln Sie ein System, das Gesprächsverläufe speichert und automatisch eine zusammenfassende Chat-Notiz erstellt. Nutzen Sie z.B. ConversationSummaryMemory, um die wichtigsten Punkte aus langen Gesprächen zu extrahieren.



<p><font color='black' size="5">
Virtuelle Assistenten mit eigenem Gedächtnis
</font></p>


Eine Datei wird bereitgestellt, die ein Gespräch zwischen zwei virtuellen Assistenten enthält. Jede Antwort soll aus genau einem Satz bestehen. Die Datei ist folgendermaßen aufgebaut:

|Assistent|Eingabe|
|---|---|
|botA|Ich heiße Max.|
|botB|Mein Name ist Emma.|
|botA|Ich wohne in Berlin.|
|botB|Ich lebe in Hamburg.|
|botA|Wo wohne ich und wie heiße ich?|
|botB|Wo lebe ich?|

**Aufgabe**  
Schreiben Sie ein Programm, das zwei virtuelle Assistenten simuliert, die jeweils ein eigenes Gedächtnis haben. Die Eingaben aus der Datei werden an den jeweiligen Assistenten geschickt, und ihre Antworten sollen in einer neuen Datei gespeichert werden.

**Erwartete Ausgabe**  
Die erwartete Ausgabe könnte wie folgt aussehen:

|Assistent|Antwort|
|---|---|
|botA|Hallo Max.|
|botB|Hallo Emma.|
|botA|Danke für diese Information.|
|botB|Danke für diese Information.|
|botA|Hallo Max, du wohnst in Berlin und dein Name ist Max.|
|botB|Hallo Emma, du lebst in Hamburg.|

**Hinweise**

- Die Antworten müssen nicht exakt mit der Beispielausgabe übereinstimmen, sollten aber sinngemäß ähnlich sein.
- Jeder Assistent hat ein eigenes Gedächtnis und sollte sich nur an seine eigenen Informationen erinnern.
- Das Programm soll die Antworten in einer neuen Datei im gleichen Tabellenformat speichern.